# Load Data

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
import numpy as np
import os

nama_folder = r"FINAL_clean binary\npy"
# for subdir, dirs, files in os.walk(nama_folder):
#     for file in files:
#         res = numpy.load(os.path.join(subdir, file))
#         print(os.path.join(subdir, file))
#         for i in res:
#             print(i)
#         print("\n\n")

# res = np.load("kamar_webcamlogi/context_data/normal_2.avi.npy")
# print(res.shape)
# # print(os.path.join("context_cek algoritma/context_data_shoplift/0.avi.npy"))
# for i in res:
#     print(i)



# Jumlah klip utk per class
no_sequences = 111
# Panjang total sliding window utk masuk model sequence classification
sequence_length = 60

# Actions that we try to detect
actions = np.array(['normal', 'shoplifting'])

label_map = {label:num for num, label in enumerate(actions)}

print(label_map)

sequences, labels = [], []
for action in actions:
    for num in range(1,no_sequences+1):
        window = np.load(os.path.join(nama_folder, action, " ({}).npy".format(num)))
        window = window.astype('float64') 

        window_holder = []
        for idx_frame, per_frame in enumerate(window):
            if(idx_frame+1)%2==0:
                per_frame_holder = []
                for i in range(len(per_frame)):
                    if(i in [0,2,4,6,8,10,12,14]):
                        if(per_frame[i] == 0):
                            per_frame_holder.append(float(-1))
                        else:
                            per_frame_holder.append(float((per_frame[i]/640)-0.5))
                    else:
                        if(per_frame[i] == 0):
                            per_frame_holder.append(float(-1))
                        else:
                            per_frame_holder.append((per_frame[i]/480)-0.5)
                window_holder.append(per_frame_holder)

        # for per_frame in window:
        #     for i in range(len(per_frame)):
        #         if(i in [0,2,4,6,8,10,12,14]):
        #             if(per_frame[i] == 0):
        #                 per_frame[i] = -1
        #             else:
        #                 per_frame[i] = float((per_frame[i]/640)-0.5)
        #         else:
        #             if(per_frame[i] == 0):
        #                 per_frame[i] = -1
        #             else:
        #                 per_frame[i] = float((per_frame[i]/480)-0.5)
            # print(per_frame)
        sequences.append(window_holder)
        labels.append(label_map[action])

# print("\n\nSEQUENCE")
# print(np.array(sequences).shape)
# print(sequences)

# print("\n\nLABELS")
# print(np.array(labels).shape)
# print(labels)

X = np.array(sequences)
y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



{'normal': 0, 'shoplifting': 1}


In [13]:
X.shape

(222, 25, 18)

# Build and Train

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard


log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


model = Sequential()
model.add(LSTM(8, return_sequences=True, activation='relu', input_shape=(25,18)))
model.add(LSTM(4, return_sequences=False, activation='relu'))
# model.add(LSTM(4, return_sequences=False, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(8, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# num_classes = 2

# def make_model(input_shape):
#     input_layer = keras.layers.Input(input_shape)

#     conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
#     conv1 = keras.layers.BatchNormalization()(conv1)
#     conv1 = keras.layers.ReLU()(conv1)

#     conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
#     conv2 = keras.layers.BatchNormalization()(conv2)
#     conv2 = keras.layers.ReLU()(conv2)

#     conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
#     conv3 = keras.layers.BatchNormalization()(conv3)
#     conv3 = keras.layers.ReLU()(conv3)

#     gap = keras.layers.GlobalAveragePooling1D()(conv3)

#     output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

#     return keras.models.Model(inputs=input_layer, outputs=output_layer)


# model = make_model(input_shape=X_train.shape[1:])


model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


In [46]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback], validation_data=(X_test, y_test))

Epoch 1/100
6/6 [==============================] - 2s 81ms/step - loss: 0.7839 - categorical_accuracy: 0.3842 - val_loss: 0.6741 - val_categorical_accuracy: 0.5333
Epoch 2/100
6/6 [==============================] - 0s 17ms/step - loss: 0.7191 - categorical_accuracy: 0.4011 - val_loss: 0.6359 - val_categorical_accuracy: 0.5111
Epoch 3/100
6/6 [==============================] - 0s 17ms/step - loss: 0.6786 - categorical_accuracy: 0.4237 - val_loss: 0.6069 - val_categorical_accuracy: 0.4889
Epoch 4/100
6/6 [==============================] - 0s 18ms/step - loss: 0.6402 - categorical_accuracy: 0.5650 - val_loss: 0.5852 - val_categorical_accuracy: 0.6444
Epoch 5/100
6/6 [==============================] - 0s 18ms/step - loss: 0.6117 - categorical_accuracy: 0.7119 - val_loss: 0.5659 - val_categorical_accuracy: 0.6889
Epoch 6/100
6/6 [==============================] - 0s 17ms/step - loss: 0.5830 - categorical_accuracy: 0.7910 - val_loss: 0.5496 - val_categorical_accuracy: 0.6667
Epoch 7/100
6/6 

In [47]:
print(model.summary())

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (None, 32)                6528      
                                                                 
 dense_19 (Dense)            (None, 2)                 66        
                                                                 
Total params: 6594 (25.76 KB)
Trainable params: 6594 (25.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [48]:
model.save('FINAL_LSTM_v1_12.5FPS_LSTM32UNIT.h5')
# model.save('FCN_V2_10FPS.h5')

c:\Users\faris\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
from tensorflow import lite
# Convert the model.
converter = lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [lite.OpsSet.TFLITE_BUILTINS,
lite.OpsSet.SELECT_TF_OPS]

tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\faris\AppData\Local\Temp\tmp8bo0igve\assets


INFO:tensorflow:Assets written to: C:\Users\faris\AppData\Local\Temp\tmp8bo0igve\assets


In [97]:
res = model.predict(X_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


In [98]:
for i in range(len(res)):
    print(actions[np.argmax(res[i])])

shoplifting
normal
normal
normal
normal
shoplifting
normal
normal
normal
shoplifting
normal
normal
shoplifting
shoplifting
shoplifting
normal
normal
normal
normal
shoplifting
normal
shoplifting
normal
shoplifting
shoplifting
shoplifting
shoplifting
normal
shoplifting
shoplifting
shoplifting
normal
shoplifting
normal
shoplifting
shoplifting
shoplifting
normal
shoplifting


In [99]:
for i in range(len(res)):
    print(actions[np.argmax(y_test[i])])

shoplifting
normal
normal
normal
normal
shoplifting
normal
normal
normal
normal
normal
normal
shoplifting
shoplifting
shoplifting
normal
normal
normal
normal
shoplifting
normal
shoplifting
normal
shoplifting
shoplifting
shoplifting
shoplifting
normal
shoplifting
shoplifting
shoplifting
normal
shoplifting
normal
shoplifting
shoplifting
shoplifting
normal
shoplifting
